## Smart Contracts and Payments in Algos
#### 07.4 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-09

* Payment transactions inside smart contacts
* Inner transactions


## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding the code below

In [ ]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

## Social security: a stateful smart contract with blockchain lookup
* Idea: check account holdings and give money to the "poor"
* The limit for "poor" is stored in `PoorLimit`
* The payout is stored in `Payout`

#### Now you do ...
* Check the holdings of Bob and Charlie
* Set the `PoorLimit` so that it is approx. 1 ALGO more than the poorest of the three

In [ ]:
asset_holdings_df(algod_client,Bob['public'])

In [ ]:
asset_holdings_df(algod_client,Charlie['public'])

#### Step 1: Define Approval program

In [ ]:
PoorLimit = Int(int( 18*1E6 ))    # <------ UPDATE!!!!!! Below XX Algos, you are "poor"
Payout   = Int(int( 0.5*1E6 ))    

In [ ]:
handle_creation = Seq(
    [
        # Initialize GLOBAL variables
        App.globalPut(Bytes("Info"),       Bytes("www.usi.ch")),
        App.globalPut(Bytes("Funds"),      Balance(Global.current_application_address())),            
        App.globalPut(Bytes("Fans"),       Int(0)),              # so far, 0 fans
        App.globalPut(Bytes("All_visits"), Int(0)),              # so far, 0 visits
        Return(Int(1)),                                          # Return "OK"
    ]
)

# Optin
fans = App.globalGet(Bytes("Fans"))
visits = App.globalGet(Bytes("All_visits"))
handle_optin = Seq (
    [
        App.globalPut(Bytes("Fans"), fans + Int(1)),                         # Add 1 to number of fans (global)
        App.localPut(Int(0), Bytes("Visits"), Int(0)),                       # Add 1 to number of visits (local)
        App.localPut(Int(0), Bytes("YourHolding"), Balance(Txn.sender())),
        Return(Int(1))
    ]
)


visits_loc  = App.localGet(Int(0), Bytes("Visits"))
handle_closeout = Seq(
    [
        App.globalPut(Bytes("Fans"), fans - Int(1)),
        App.globalPut(Bytes("All_visits"), visits - visits_loc),
        Return(Int(1)),
    ]
)

handle_updateapp = Return( 
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

# handle interaction
visits_loc  = App.localGet(Int(0), Bytes("Visits"))
holdings_loc  = App.localGet(Int(0), Bytes("YourHolding"))
handle_noop = Seq(
    [
        App.globalPut(Bytes("All_visits"), visits+Int(1)),
        App.localPut(Int(0), Bytes("Visits"), visits_loc+Int(1)),
        App.localPut(Int(0), Bytes("LastHolding"), holdings_loc),
        If(Balance(Txn.sender()) < PoorLimit,
           # if-part
            Seq([
                InnerTxnBuilder.Begin(),
                InnerTxnBuilder.SetFields({
                    TxnField.type_enum: TxnType.Payment,
                    TxnField.sender: Global.current_application_address(),
                    TxnField.amount: Payout,
                    TxnField.receiver: Txn.sender()
                    }),
                InnerTxnBuilder.Submit(),
                App.localPut(Int(0), Bytes("Message"), Bytes("You are welcome"))   
                ]),  
            # end if-part
            # else
            App.localPut(Int(0), Bytes("Message"), Bytes("You have enough"))   
            ),
            # end else-part
        App.globalPut(Bytes("Funds"),     Balance(Global.current_application_address())),                    
        App.localPut(Int(0), Bytes("YourHolding"), Balance(Txn.sender())),
        Return(Int(1))
    ]
)

In [ ]:
social_approval_pyteal = Cond(
    [Txn.application_id() == Int(0),              handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,    handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut, handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp, handle_noop],
)

#### Compile PyTEAL -> TEAL
* Notice the `Mode.Application` (was `Mode.Signature`)

In [ ]:
social_approval_teal = compileTeal(social_approval_pyteal,mode=Mode.Application, version=5)
#print(social_approval_teal)

#### Step 1b: Define Clear State program
* This program handles forced opt-outs

In [ ]:
visits = App.globalGet(Bytes("All_visits"))
visits_loc  = App.localGet(Int(0), Bytes("Visits"))

social_clear_pyteal = Seq(
    [
        App.globalPut(Bytes("All_visits"), visits - visits_loc),
        Return(Int(1)),
    ]
)

In [ ]:
social_clear_teal = compileTeal(social_clear_pyteal,mode=Mode.Application, version=3)
print(social_clear_teal)

#### Compile TEAL -> Bytecode
This is slightly different ... we need one additional step for Byte-encoding

In [ ]:
social_approval_b64 = algod_client.compile(social_approval_teal)
Social_Approval =  base64.b64decode(social_approval_b64['result'])

social_clear_b64 = algod_client.compile(social_clear_teal)
Social_Clear =  base64.b64decode(social_clear_b64['result'])

## Deploy Smart Contract

##### Alice deploys the smart contract
* Reserve (global/local) storage with `StateSchema`

In [ ]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()
creator = Alice

# How much space do we need?
global_ints = 3    # for Fans, All_Visits, Funds
global_bytes = 1   # for Info
social_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 3     # For Visits, YourHolding, LastHolding
local_bytes = 1    # For Message
social_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = creator['public'],           # <-- sender public
      sp = sp,                              # <-- sp
      on_complete = 0,                      # <- what to do when finished (nothing)
      approval_program = Social_Approval,   # <-- approval program 
      clear_program = Social_Clear,         # <-- clear program 
      global_schema = social_global_schema, # <-- reserve global space 
      local_schema = social_local_schema    # <-- reserve local space
    )

In [ ]:
# Step 2-4: sign, send, wait
stxn = txn.sign(creator['private'])
txid=algod_client.send_transactions([stxn])
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

## The Smart Contract is now deployed
* And there is alreasdy something to see

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
# Program code immediately visible on the web
print('https://testnet.algoexplorer.io/application/{}'.format(app_id))

## To fund the Smart Contract, we need its address
There is a difference

* `app_id` is the ID number of the Smart Contract, it is needed to call it via `ApplicationNoOpTxn()`
* `app_addr` is the public address of the Smart contract, it is needed to fund it via a `PaymentTxn()`


In [ ]:
import algosdk.logic
app_addr = algosdk.logic.get_application_address(app_id)
print(app_addr)

In [ ]:
# Fund the smart contact via the Algorand Dispenser
print('https://dispenser.testnet.aws.algodev.network/?account='+app_addr)

In [ ]:
# Check holdings of Smart Contract
asset_holdings_df(algod_client,app_addr)

## Using the Smart Contract (1): Users opt-in
* No money is dispensed here

In [ ]:
user = Bob

# Step 1: prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)

# Step 2: sign transaction
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Watch the state of the contract evolve

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,user['public'],app_id)

#### Now try
* For Charlie to opt in

## Using the Smart Contract (2): Call Smart Contract

* Smart Contract checks holdings
* Disperses 1/2 Algo, if caller is below limit

In [ ]:
user = Bob

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(user['public'], sp, app_id)

# Step 2+3: sign and send
stxn = txn.sign(user['private'])
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Watch the state of the contract evolve

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,user['public'],app_id)

#### Now try
* Repeatedly call the Smart Contract from an account that is below the limit
* Call the Smart Contract from an account that is above the limit

## Users close out (leave) App
* With a `ApplicationCloseOutTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCloseOutTxn)

In [ ]:
user = Bob

# Step 1-4
sp = algod_client.suggested_params()
txn = transaction.ApplicationCloseOutTxn(user['public'], sp, app_id)
stxn = txn.sign(user['private'])
txid = algod_client.send_transactions([stxn])
txinfo = wait_for_confirmation(algod_client, txid)

#### Watch the state of the contract evolve

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,user['public'],app_id)

#### Now try
* Charlie should also close out

## Deleting the app
* Rather important, because an address can only create **10 apps**
* App can be deleted by creator

In [ ]:
creator = Alice

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])